# K-Nearest Neighbors (KNN) in Python #

Killian McKee

1. [What is KNN?](#section1)
2. [Key Terms](#section2) 
3. [Pros and Cons KNN](#section3)
4. [When to use KNN](#section4)
5. [Key Parameters](#section5) 
6. [Walkthrough: Building a KNN Classifier](#section6)
7. [Additional Resources](#section7) 
8. [Conclusion](#section8)
9. [Sources](#section9)

<a id='section1'></a> 

## What is KNN?##

K-nearest neighbors is a supervised, non-parametric classification algorithm. In other words, knn attempts to classify things into different pre-defined groups without making assumptions about the underlying shape of the data. Knn works by comparing the distance (euclidean, manhattan, etc.) of new data points to all existing data points. The algorithm then selects the k-nearest data points where k can be any integer (selected by the user). Lastly, the class to which the majority of the k nearest points belong is assigned to the new data point. We can see in the image below how changing the value of k can impact classification results; when k=1 the new example to classify (denoted by the green circle) would be put into class 1 since its only neighbor is in class 1 (blue square), but if k is increased to 3 then the new classification changes to class 2 since the triangle count>square count for the new circle. Using a larger value of k will lead to more things being classified as the most common class at the cost of capturing intricate decision boundaries. Conversely, small k values are more adversely affected by noise or variance in the data, but can capture variable shapes more accurately. Typically, optimal k-values are found by performing a train/test split on the data and then assessing model accuracy with different levels of k. 

<img src='knn.png'>

<a id='section2'></a> 

### Key Terms ###

1. **k**: a numerical value representing the distance (euclidean, manhattan, etc.) within which the knn model will look for neighbors to use in classification.
2. **non-parametric**: makes no assumptions about the underlying data shape/distribution
3. **clustering**: forming non-overlapping groups of similar objects

<a id='section3'></a> 

### Pros and Cons of KNN ###

#### Pros ####
1. **Easy to Implement**: knn is a straightforward algorithm that is easy to implement, visualize, and understand. 
2. **No Required Training**:knn is a [lazy learning](https://en.wikipedia.org/wiki/Lazy_learning) algorithm, which means it doesn't need to train on data ahead of time to make new classifications. This means knn is very fast on small/medium sized datasets. 
3. **New Data Integrated Seamlessly**: since no data is needed for training, new data can be integrated more seamlessly into knn than most other clustering algorithms. 

#### Cons #### 
1. **HDD Shortcomings**: high dimesnional data (hdd), or data with many features can make it difficult for the knn algorithm to measure distances between points
2. **Slow on Large Datasets**: knn does not scale up to very large datasets well. This is because it becomes increasingly costly to measure the distance between data points as the number of points increases. 
3. **Categorical Shortcomings**: knn doesn't handle categorical features well because it seeks works by comparing the distance between features. 

<a id='section4'></a> 

### When to use KNN ###

Knn works well at performing non-parametric clustering and classifications on small/medium sized datasets in which most of the features are not categorical. It is especially useful when one wants to continuously generate new classifications since it is a lazy learner.

<a id='section5'></a> 

### Key Parameters ###

**n_neighbors**: the number of neighbors to include in the classificiation
**weights**: weight function used in prediction. Allows some points to be weighted more heavily than others e.g. close points are more important that far ones, etc. [More info](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)
**algorithm**: algorithm used to compute nearest neighbor. "Auto" is the most common choice because it attempts to find the best fit out of all the available options. 

<a id='section6'></a> 

### Walkthrough: Building a KNN Classifier ###

This walkthrough steps through how to build a classifier on the [iris dataset](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html). We will be attempting classify new observations as one of three possible iris species based on factors like their petal length and width

In [61]:
# import necessary packages 

import pandas as pd 
import numpy as np 
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris

In [50]:
# load in the data and convert it from an array into a dataframe
# the conversion is optional, but most datasets won't come in the scikit array format

data=load_iris()
df = pd.DataFrame(data['data'], columns=data['feature_names'])
df['target'] = data['target']

In [51]:
# examine the top of the dataset 

df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [52]:
# examine the shape of the data 
# 150 rows with 5 columns 

df.shape

(150, 5)

In [53]:
# split our dataset into data and target (x&y)

X=df.drop(columns='target')
y=df['target']

In [54]:
# check our x dataset

X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [55]:
# check our y dataset 

y.head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int32

In [60]:
# we will be building and validating our model with kfold cross validation, which tends to be the most accurate method

#initialize the model 
knn_cv = KNeighborsClassifier(n_neighbors=3,weights='uniform', algorithm='auto')

#train model with cv of 5 
cv_scores = cross_val_score(knn_cv, X, y, cv=5)

#print each cv score (accuracy) and average them
print(cv_scores)
print('cv_scores mean:{}'.format(np.mean(cv_scores)))

[0.96666667 0.96666667 0.93333333 0.96666667 1.        ]
cv_scores mean:0.9666666666666668


In [63]:
# although we achieved an accuracy of 96%, lets do some hyperparameter tuning to find an optimal k value 
# gridsearchCV is an optimization method we can use to find the best parameters for a model 

#create new a knn model
knn2 = KNeighborsClassifier()

#create a dictionary of all values we want to test for n_neighbors
param_grid = {'n_neighbors': np.arange(1, 20)}

#use gridsearch to test all values for n_neighbors
knn_gscv = GridSearchCV(knn2, param_grid, cv=5)

#fit model to data
knn_gscv.fit(X, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [71]:
# display the best parameters for the model 

print('optimal neighbor parameter:',knn_gscv.best_params_)


optimal neighbor parameter: {'n_neighbors': 6}


In [74]:
# display the best accuracy achieved by our gridsearch model 

print('best accuracy:',knn_gscv.best_score_)

best accuracy: 0.98


<a id='section7'></a> 

### Additional Resources ###

1. [Knn video explanation](https://www.youtube.com/watch?v=UqYde-LULfs)
2. [Knn from scratch](https://medium.com/deep-math-machine-learning-ai/chapter-5-k-nearest-neighbors-algorithm-with-code-from-scratch-7f93f653c860)

<a id='section8'></a> 

### Conclusion ###

In this tutorial we stepped through knn, which is a supervised learning clustering algorithm that measures the distance between a new point and its nearest neighbors to form clusters. We also covered some of the pros and cons i.e. learned that it is a lazy learner, meaning it can quickly integrate new data without training, but that it struggles with categorical and high dimensional data. Next, we tackled use cases, parameters, and a walkthrough on how to implement a knn classifier with scikit learn. Those looking to go into greater depth should check out the knn from scratch link above, which steps through how to build a knn model without scikit. 

<a id='section9'></a> 

### Sources ###
1. https://stackabuse.com/k-nearest-neighbors-algorithm-in-python-and-scikit-learn/
2. https://medium.com/@adi.bronshtein/a-quick-introduction-to-k-nearest-neighbors-algorithm-62214cea29c7
3. https://towardsdatascience.com/building-a-k-nearest-neighbors-k-nn-model-with-scikit-learn-51209555453a
4. https://medium.com/deep-math-machine-learning-ai/chapter-5-k-nearest-neighbors-algorithm-with-code-from-scratch-7f93f653c860